# Bubble plot

Use this notebook to build a new `cordex_overview.csv` from the specified CORDEX netCDF dataset.

In [1]:
from ruins.core import build_config
from collections import defaultdict
import plotly.graph_objects as go
import pandas as pd

Define the DATA_FILE here, without file ending.

In [2]:
DATA_FILE = 'cordex_krummh'
config, dm = build_config()


This cell will write the new CSV file into the data folder. Keep in mind, that you still have to push it into the data zip on Zenodo.

In [7]:
# get krummh models
krumm = dm[DATA_FILE].read()
for v in krumm.data_vars:
    print(krumm[v].attrs)
    break

# build the structure
combinations = [krumm[v].attrs for v in krumm.data_vars]
N = len(combinations)

# aggregate
agg = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0)))

for c in combinations:
    agg[c['GCM']][c['RCM']][c['RCP']] += 1

list(agg.keys())

ids = ["RUINS"]
labels = ["RUINS"]
parents = [""]
values = [N]
customdata = [('100% of RUINS', '-',)]
colors = [""]

# sum([sum([_ for _ in v.values()]) for v in agg['CanESM2'].values()])
# add the gcms
for gcm in agg.keys():
    ids.append(f'RUINS.{gcm}')
    labels.append(gcm)
    parents.append('RUINS')
    gcm_val = sum([sum([r for r in rcm.values()]) for rcm in agg[gcm].values()])
    values.append(gcm_val)
    customdata.append((f'{int(gcm_val / N * 100)}% of RUINS', '-',))

    # add the RCMs
    for rcm in agg[gcm].keys():
        ids.append(f'RUINS.{gcm}.{rcm}')
        labels.append(rcm)
        parents.append(f'RUINS.{gcm}')
        rcm_val = sum([r for r in agg[gcm][rcm].values()])
        values.append(rcm_val)
        customdata.append((f'{int(rcm_val / N * 100)}% of RUINS' ,f'{int(rcm_val / gcm_val * 100)}% of {gcm}' ,))

        # add the RCPs
        for rcp in agg[gcm][rcm].keys():
            ids.append(f'RUINS.{gcm}.{rcm}.{rcp}')
            labels.append(rcp.upper())
            parents.append(f'RUINS.{gcm}.{rcm}')
            val = agg[gcm][rcm][rcp]
            values.append(val)
            customdata.append((f'{int(val / N * 100)}% of RUINS', f'{int(val / rcm_val * 100)}% of {rcm}',))

df = pd.DataFrame(dict(ids=ids, labels=labels, parents=parents, values=values, customdata1=[_[0] for _ in customdata], customdata2=[_[1] for _ in customdata]))
#df.to_csv('../data/cordex_overview.csv', index=None)

{'GCM': 'HadGEM2-ES', 'RCM': 'COSMO-crCLIM-v1-1', 'RCP': 'rcp85', 'ensemble': 'r1i1p1', 'institute': 'CLMcom-ETH'}


## TEST

This is for testing purposes only: This is a clone of the new `ruins.plotting.sunburst` function

In [8]:
df = pd.read_csv('../data/cordex_overview.csv')
ids = df.ids.values
labels = df.labels.values
parents = df.parents.values
values = df['values'].values
customdata = list(zip(df.customdata1.values, df.customdata2.values))

fig = go.Figure()

fig.add_trace(go.Sunburst(
    ids=ids,
    labels=labels,
    parents=parents,
    values=values,
    branchvalues='total',
    insidetextorientation='radial',
    #hoverinfo='label+percent root+percent parent+value',
    maxdepth=4,
    customdata=customdata,
    hovertemplate='<b>Name</b>: %{label}<br><b>Total</b>: %{value}<br><br>%{customdata[0]}<br>%{customdata[1]}',
    name='',
    marker=dict(colorscale='Electric')
))
fig.update_layout(margin=dict(t=0, l=0, r=0, b=0),  width=700, height=700)
fig